In [25]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnableLambda, RunnableMap

chat = ChatOpenAI(temperature=0.1)

prouduct_prompt = ChatPromptTemplate.from_messages([
  ("system", "you are a product manager. you will help to build a ai product.if you are not sure about something say 'i am not sure about that'"),
  ("human", "{user_goal_prompt}"),
])
prouduct_chain = prouduct_prompt | chat

In [26]:
design_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a product designer. You will help design the AI product. If you're not sure about something, say 'I'm not sure about that'."),
    ("human", "{product}"),
])
design_chain = design_prompt | chat

In [27]:
devolper_prompt = ChatPromptTemplate.from_messages([
  ("system","You are a developer. You will write code for the AI product based on product and design. If you're not sure about something, say 'I'm not sure about that'."),
  ("human","Product Info:\n{product}\n\nDesign Info:\n{design}"),
]
)
devolper_chain = devolper_prompt| chat

In [28]:
def full_chain(user_goal_prompt: str):
  product = prouduct_chain.invoke({"user_goal_prompt": user_goal_prompt}).content
  design = design_chain.invoke({"product": product}).content
  code = devolper_chain.invoke({"product": product, "design": design}).content
  return {
      "product": product,
      "design": design,
      "code": code
  }

In [29]:
result = full_chain("I want to build a decision-making AI agent that helps our sales team determine whether a merchant is eligible for a fee discount.The agent should read our internal policy documents and, when the sales team types in a merchant name and case, the agent should return a yes/no answer with explanation.The goal is to avoid manual back-and-forth between sales and operations.")
print("🔷 Product:\n", result["product"])
print("\n🎨 Design:\n", result["design"])
print("\n💻 Code:\n", result["code"])

🔷 Product:
 To build a decision-making AI agent for this purpose, we can follow these steps:

1. Data Collection: Gather all relevant internal policy documents that outline the criteria for fee discounts. This data will be used to train the AI agent.

2. Data Preprocessing: Clean and preprocess the data to make it suitable for training the AI model. This may involve tasks such as text normalization, tokenization, and removing irrelevant information.

3. Model Selection: Choose a suitable machine learning model for text classification tasks. Models like Natural Language Processing (NLP) models such as BERT or LSTM can be effective for this purpose.

4. Training: Train the selected model on the preprocessed data to learn the patterns and criteria for determining fee discounts eligibility.

5. Integration: Develop an interface where the sales team can input the merchant name and case details. The AI agent will then process this input and provide a yes/no answer along with an explanation b